In [12]:
kajal = True
try:
    module("unload", "cuda/11.6")
    module("load", "cuda/11.4")
    module("load","ffmpeg")
    module("list")
except: 
    kajal = False

In [2]:
import sys
from pathlib import Path
if kajal:
    path_root = Path( '/cobra/u/kkumari/warp-drive')
    sys.path.append(str(path_root))

else:
    path_root = Path( '/project_ghent/MARL/warp-drive/')
    sys.path.append(str(path_root))
    path_root = Path( '/project_ghent/MARL/collective_MARL/')
    sys.path.append(str(path_root))
print(sys.path)

['/project_ghent/MARL/collective_MARL', '/project_ghent/MARL/MARL_env/lib/python38.zip', '/project_ghent/MARL/MARL_env/lib/python3.8', '/project_ghent/MARL/MARL_env/lib/python3.8/lib-dynload', '', '/project_ghent/MARL/MARL_env/lib/python3.8/site-packages', '/project_ghent/MARL/warp-drive', '/project_ghent/MARL/collective_MARL']


In [3]:
# Log in to your W&B account
import wandb
"90dc2cefddde123eaac0caae90161981ed969abe"
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/tcazalet/.netrc


True

In [4]:
import torch
# assert torch.cuda.device_count() > 0, "This notebook needs a GPU to run!"

from importlib import reload

import logging
# Set logger level e.g., DEBUG, INFO, WARNING, ERROR
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

# Load the run config.
from config import run_config

/project_ghent/MARL/MARL_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from warp_drive.utils.env_registrar import EnvironmentRegistrar
from custom_env import CUDACustomEnv

env_registrar = EnvironmentRegistrar()
env_registrar.add_cuda_env_src_path(CUDACustomEnv.name, "custom_env_step_numba", env_backend="numba")

from warp_drive.env_wrapper import EnvWrapper
from warp_drive.training.utils.device_child_process.child_process_base import event_messenger

env_wrapper = EnvWrapper(
    env_obj=CUDACustomEnv(**run_config["env"]),
    num_envs=run_config["trainer"]["num_envs"],
    env_backend="numba",
    env_registrar=env_registrar,
    event_messenger=event_messenger,
    process_id=0
)

from warp_drive.training.models.factory import ModelFactory

model_factory = ModelFactory.add("predator_policy", "predator_policy", "PredatorPolicy") 
model_factory = ModelFactory.add("prey_policy", "prey_policy", "PreyPolicy") 


/project_ghent/MARL/MARL_env/lib/python3.8/site-packages/numba/cuda/random.py:45: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(forceobj=_forceobj, looplift=_looplift)
/project_ghent/MARL/MARL_env/lib/python3.8/site-packages/numba/cuda/random.py:71: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(forceobj=_forceobj, looplift=_looplift)
/project_ghent/MAR

function_manager: Setting Numba to use CUDA device 0


In [6]:
policy_tag_to_agent_id_map = {
    "predator": list(env_wrapper.env.predators),
    "prey": list(env_wrapper.env.preys),
}

In [7]:
from warp_drive.training.trainer_wandb import Trainer

trainer = Trainer(
    env_wrapper=env_wrapper,
    config=run_config,
    policy_tag_to_agent_id_map=policy_tag_to_agent_id_map,
    device_id=0,
    num_devices=1,
    verbose=False
)
trainer.train()

wandb: Currently logged in as: tanguy-cazalets (tcazalet_airo). Use `wandb login --relogin` to force relogin
/project_ghent/MARL/MARL_env/lib/python3.8/site-packages/wandb/sdk/lib/ipython.py:70: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


In [8]:
import os

name = run_config["saving"]["name"]
tag = run_config["saving"]["tag"]

directory = f"/tmp/{name}/{tag}"
files = os.listdir(directory)
max_number = max([int(number) for number in files])

total_lenght = run_config["env"]["episode_length"]*run_config["trainer"]["num_episodes"]

trainer.load_model_checkpoint(
    {
        "prey": f"/tmp/{name}/{tag}/{max_number}/prey_{total_lenght}.state_dict",
        "predator": f"/tmp/{name}/{tag}/{max_number}/predator_{total_lenght}.state_dict",
    }
)

In [9]:
import animations
reload(animations)
from animations import generate_animation
from IPython.display import HTML

predator_size = run_config["env"]["predator_size"]
prey_size = run_config["env"]["prey_size"]

# Visualize the entire episode roll-out
anim = generate_animation(trainer, predator_size=predator_size, prey_size=prey_size)

In [10]:
HTML(anim.to_html5_video())

In [11]:
# Close the trainer to clear up the CUDA memory heap
trainer.graceful_close()